In [9]:
#the pix4D pointcloud problem...

#where the pix4D cloud overlaps with the SE cloud, there is a variable vertical offset....
#I want to write code that will: (1) merge the two clouds (2) determine where they overlap (3) determine what the vertical offset is for areas that overlap
#... (4) shift the pix4d cloud by the identified offset, and (5) export the adjusted pix4d cloud so that....
    # so that we can then resample both of the clouds in SE and export them as one cloud

#first problem: I need to crop the SE point cloud so that it is only the area closest to the pix4dcloud? This way I can work with a smaller file size.

import subprocess
import numpy as np
import os

# Define file paths
cloud1_file = r"D:\cropped cloud\cropped_cloud.las" # Path to the first .laz file (reference)
cloud2_file = r"C:\Users\olivi\Desktop\Pix4Dmatic processing\sept_15_flt_2_missing_data\exports\sept_15_flt_2_missing_data-dense_point_cloud.las" # Path to the second .laz file (to be adjusted)
merged_file = "merged.laz"  # Temporary merged file
output_file = "adjusted_sept_15_flt_2_missing_data_chunk_E.laz"  # Output adjusted .laz file

# Ensure LAStools executable path is set (update if necessary)
lasmerge_path = r"C:\lastools\bin\lasmerge.exe"  # Update to the location of lasmerge.exe
lasdiff_path = r"C:\lastools\bin\lasdiff.exe"  # Update to the location of lasdiff.exe
lasheight_path = r"C:\lastools\bin\lasheight.exe"  # Update to the location of lasheight.exe
las2las_path = r"C:\lastools\bin\las2las.exe"  # Update to the location of las2las.exe

# Step 1: Merge the two point clouds to identify overlapping areas
merge_command = [
    lasmerge_path,
    "-i", cloud1_file, cloud2_file,
    "-o", merged_file
]

try:
    subprocess.run(merge_command, check=True)
    print(f"Merged file created: {merged_file}")
except subprocess.CalledProcessError as e:
    print(f"Error during merging: {e}")
    raise

# Step 2: Identify vertical differences in the overlapping areas
diff_command = [
    lasdiff_path,
    "-i", f"\"{cloud1_file}\"",  # Add quotes around the paths
    f"\"{cloud2_file}\"",
    "--vertical"
]

try:
    result = subprocess.run(diff_command, capture_output=True, text=True, check=True)
    output = result.stdout
    print("Vertical differences calculated.")
    
    # Parse the vertical offset from the output (example assumes average is reported)
    lines = output.splitlines()
    offset_line = [line for line in lines if "average z difference" in line]
    if offset_line:
        vertical_offset = float(offset_line[0].split()[-1])
        print(f"Vertical offset identified: {vertical_offset} meters")
    else:
        raise ValueError("Could not parse vertical offset from lasdiff output.")
except subprocess.CalledProcessError as e:
    print(f"Error during vertical difference calculation: {e}")
    print(f"Standard output: {e.stdout}")  # Print stdout
    print(f"Standard error: {e.stderr}")  # Print stderr
    raise


# Step 3: Adjust the second point cloud by the vertical offset
adjust_command = [
    las2las_path,
    "-i", cloud2_file,
    "-o", output_file,
    "--translate_z", str(vertical_offset)
]

try:
    subprocess.run(adjust_command, check=True)
    print(f"Adjusted point cloud saved as: {output_file}")
except subprocess.CalledProcessError as e:
    print(f"Error during vertical adjustment: {e}")
    raise


Merged file created: merged.laz
Error during vertical difference calculation: Command '['C:\\lastools\\bin\\lasdiff.exe', '-i', '"D:\\cropped cloud\\cropped_cloud.las"', '"C:\\Users\\olivi\\Desktop\\Pix4Dmatic processing\\sept_15_flt_2_missing_data\\exports\\sept_15_flt_2_missing_data-dense_point_cloud.las"', '--vertical']' returned non-zero exit status 1.
Standard output: 
Standard error: ERROR: cannot understand argument '--vertical'



CalledProcessError: Command '['C:\\lastools\\bin\\lasdiff.exe', '-i', '"D:\\cropped cloud\\cropped_cloud.las"', '"C:\\Users\\olivi\\Desktop\\Pix4Dmatic processing\\sept_15_flt_2_missing_data\\exports\\sept_15_flt_2_missing_data-dense_point_cloud.las"', '--vertical']' returned non-zero exit status 1.

In [15]:
!pip install laspy
import laspy

cloud1_file = r"D:\cropped cloud\cropped_cloud.las" # Path to the first .laz file (reference)
cloud2_file = r"C:\Users\olivi\Desktop\Pix4Dmatic processing\sept_15_flt_2_missing_data\exports\sept_15_flt_2_missing_data-dense_point_cloud.las" # Path to the second .laz file (to be adjusted)

# Load the two point clouds
cloud1 = laspy.read(cloud1_file)
cloud2 = laspy.read(cloud2_file)

# Compute the vertical offset by comparing the z-coordinates
z_diff = cloud1.z - cloud2.z
mean_offset = np.mean(z_diff)
print(f"Mean vertical offset: {mean_offset} meters")

   ---------------------------------------- 0.0/84.3 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.3 kB ? eta -:--:--
   ----------------------------- ---------- 61.4/84.3 kB 825.8 kB/s eta 0:00:01
   ---------------------------------------- 84.3/84.3 kB 948.9 kB/s eta 0:00:00


ValueError: operands could not be broadcast together with shapes (534855795,) (1567796427,) 